# Ansible - Baremetal Deployment  (OneView and ILO)

created by Dirk Derichsweiler<br>
do not hesitate to contact me: derdirk@hpe.com


<img src="media/Overview.png" height="800" width="800" align="left"> <br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

---
***kudos to:***

Christopher Kurka, to provide the example code in yammer!! --> https://www.yammer.com/hpe.com/#/threads/show?threadId=415736540717056<br>
Stephan Koch, to find it and create a full working demo in the demo center in BBN!

---
## jupyter Notebook

you can find the notebook here: https://github.com/dderichswei/ansible-proliant-deployment <br>
Ansible Deployment video is hosted on youtube: https://youtu.be/antOkU74YXk



## requirements

a web-server, to host the iso and the kickstart file! 

***the following python libraries:***<br>
hpOneView           5.0.0b0 - https://github.com/HewlettPackard/oneview-ansible <br>
python-hpilo        4.3 - https://docs.ansible.com/ansible/latest/modules/hpilo_boot_module.html<br>
<br>
***Optional:***<br>
hpICsp              1.0.2 - https://github.com/HewlettPackard/python-hpICsp<br>
hpe3par-sdk         1.2.0 - https://github.com/HewlettPackard/hpe3par_ansible_module<br>
python-3parclient   4.2.9 - https://github.com/hpe-storage/python-3parclient<br>

## additional information

create a kickstart file:
https://access.redhat.com/labsinfo/kickstartconfig

On your HPE OneView appliance, or online<br>
https://10.0.20.50/api-docs/current/ <br>
http://www.hpe.com/info/oneview/docs <br>
https://developer.hpe.com/


## quick installation
- install docker:
```curl https://get.docker.com | bash ```

- get source code:
``` git clone https://github.com/dderichswei/synergy```

- start jupyter, webserver and ansible within a container: ```ddcreate.sh```


# Preparation

## Prepare Boot Media

* Download CentOS/Fedora/RHEL ISO image and mount it on any Linux Server
* cp -pR it to a local folder 
* edit the **grub.cfg** under root of the CDROM and in EFI/BOOT

```bash
menuentry 'Install CentOS 7' --class fedora --class gnu-linux --class gnu --class os {
        linuxefi /images/pxeboot/vmlinuz inst.stage2=hd:LABEL=CentOS\x207\x20x86_64 quiet inst.ks=http://osdepl.demo.local/centos/centos7ks.cfg
        initrdefi /images/pxeboot/initrd.img
        }
```

* write an new customized iso with:

```bash
mkisofs -o /tmp/centos7custom.iso -b isolinux/isolinux.bin -J -R -l -c isolinux/boot.cat -no-emul-boot -boot-load-size 4 -boot-info-table -eltorito-alt-boot -e images/efiboot.img -no-emul-boot -graft-points -V "CentOS7 Server.x86_64" .
```

* Place customized iso and kickstart file on an reachable web server
* dhcsp server must be available
* create an Server Profile Template
    with an ilo user in in
    with an network connection where the dhcp request could be handled
    and in my case with an SAN disk and boot from SAN configuration

# start deployment

## disable SSH Key Checking

to get rid of the typical ssh question:
The authenticity of host '10.0.33.66 (10.0.33.66)' can't be established.
ECDSA key fingerprint is SHA256:WdUFI0W5cfEBNMwm7tT1neBSDqCROlHC/2UATH1uAUE.
Are you sure you want to continue connecting (yes/no)?


In [1]:
export ANSIBLE_HOST_KEY_CHECKING=False
export ANSIBLE_SSH_ARGS='-o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null'

## create ansible playbook for Server Deployment for DL/ML/SYNERGY with ILO

### Workflow explanation!

http://10.0.31.73:8842/view/media/deploy-the-cool-way.png

## adapt your deployment

```
* ipaddr=<IP Adress for the deployed Server>
* hostn=<hostname of the server>
* profile_name=<OneView Server Profile Name>
* osflavor=<variable for flavor, ESX, CENTOS, RHEL...> ** # no edit required for this playbook **
* osbase=LINUXBOOT # no edit required for this playbook
* ov_template=<Server Profile Template>
* install=<path for the installation CD>
* source=<filename for the kickstart template> 
* destination=<path to the kickstart master file> # will be created
```

In [2]:
cat << \EOF > deployment-dd.yml
- hosts: localhost
  gather_facts: no
  vars_files:
    - vault.yml 
  vars:
    config: '{{ playbook_dir }}/oneview_config.json'
    server_hardware_type_name: "SY480G10 M1:416SA M3:4820CNA"
    enclosure_group_name: "3 Frame VC-ETH SAS-Switch"
    auto_assign_server_hardware: "False"
    ov_server:         "CTC H5 HE21, bay 3"

  tasks:
    - set_fact: 
        ipaddr=10.0.33.66
        hostn=dd1-ansible
        profile_name=Ansible-Deployment-for-TSS2020-Dirk
        osflavor=LINUXBOOT3 
        osbase=LINUXBOOT 
        ov_template=ANSIBLE_OS_Deploy_via_iLO 
        install=http://osdepl.demo.local/centos/centos7custom.iso 
        source=kscentos.cfg.dd 
        destination=/persistent/osdepl/centos/centos7ks.cfg

    - name: create custom kickstart file
      template:
        src: "{{source}}"
        dest:  "{{destination}}"
      delegate_to: localhost

    - name : "Create Server Profile from Template - {{ profile_name }}"
      oneview_server_profile:
        config: "{{ config }}"
        data:
          name: "{{ profile_name }}"
          serverProfileTemplateName: "{{ ov_template }}"
          serverHardwareName:         "{{ ov_server }}"
      register: mpHostInfo
      delegate_to: localhost

    - name: Task to boot a system using an ISO from an HP iLO interface only if the system is an HP server
      hpilo_boot:
        host: "{{ server_hardware.mpHostInfo.mpIpAddresses[1].address }}" 
        login: skoch
        password: "{{ iloPW }}"
        media: cdrom
        image: "{{ install }}" 
      delegate_to: localhost
        
EOF

In [3]:
ansible-playbook ./deployment-dd.yml --vault-password-file .vault_password_file

[WARNING]: No inventory was parsed, only implicit localhost is available
[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [set_fact] ****************************************************************
ok: [localhost]

TASK [create custom kickstart file] ********************************************
ok: [localhost -> localhost]

TASK [Create Server Profile from Template - Ansible-Deployment-for-TSS2020-Dirk] ***
changed: [localhost -> localhost]

TASK [Task to boot a system using an ISO from an HP iLO interface only if the system is an HP server] ***
changed: [localhost -> localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=4    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



## Deploy an application afterwards.

In [ ]:
cat << \EOF > application.yml
---
- hosts: all
  gather_facts: no
  remote_user: dderichswei
  tasks:
   - set_fact: 
        ipaddr=10.0.33.66  
   
   - name: wait for port 22 to become active
     wait_for:
        host: "{{ipaddr}}"
        port: 22
        delay: 20
        state: started
        timeout: 1600
     delegate_to: localhost


   - name: exchange public ssh key with target
     authorized_key:
       user: dderichswei
       state: present
       manage_dir: yes
       key: "{{ lookup('file', '/home/notebook/.ssh/id_rsa.pub') }}"

 
   - name: Install Nginx
     remote_user: dderichswei
     become: yes
     become_method: sudo
     docker_container:
        name: nginx
        image: nginx:latest
        detach: True
        ports:
            - 80:80
     delegate_to: "{{ ipaddr }}"

   - name: Tweak Nginx
     remote_user: dderichswei
     become: yes
     become_method: sudo
     command:  docker exec -it nginx sed -i 's/nginx/TSS2020/g' /usr/share/nginx/html/index.html
     delegate_to: "{{ ipaddr }}"

EOF

## run the playbook

In [5]:
ansible-playbook ./application.yml -i hosts --vault-password-file .vault_password_file


PLAY [all] *********************************************************************

TASK [set_fact] ****************************************************************
ok: [10.0.33.66]

TASK [wait for port 22 to become active] ***************************************
ok: [10.0.33.66 -> localhost]

TASK [exchange public ssh key with target] *************************************
changed: [10.0.33.66]

TASK [Install Nginx] ***********************************************************
changed: [10.0.33.66 -> 10.0.33.66]

TASK [Tweak Nginx] *************************************************************
changed: [10.0.33.66 -> 10.0.33.66]

PLAY RECAP *********************************************************************
10.0.33.66                 : ok=5    changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



http://10.0.33.66

# delete deployment

In [ ]:
cat << \EOF >> delete_deployment_dd.yml
EOF

# other files you might need

## create kickstart file

In [ ]:
cat << \EOF >> kscentos.cfg.dd 
lang en_US.UTF-8
keyboard de
timezone Etc/GMT+1 --isUtc
text
install
skipx
network  --bootproto=static --ip={{ipaddr}} --netmask=255.255.255.0 --gateway=10.0.33.254 --nameserver=10.0.20.5 --hostname={{hostn}}
firstboot --enable
cdrom
rootpw $1$JvvxYG0H$UfwsVbYl0R8FNoWAf8Oj01 --iscrypted
ignoredisk --only-use=/dev/disk/by-id/dm-name-mpatha
zerombr
clearpart --all --initlabel
autopart --type=lvm
auth --passalgo=sha512 --useshadow
reboot

user --name=dderichswei --password $1$JvvxYG0H$UfwsVbYl0R8FNoWAf8Oj01 --iscrypted --groups=dderichswei,wheel
#repo --name=docker --baseurl=https://download.docker.com/linux/centos/docker-ce.repo

# Disable firewall and selinux
firewall --disabled
selinux --disabled

%pre
%end

%packages
@Base
@Core
%end


%post
echo "dderichswei        ALL=(ALL)       NOPASSWD: ALL" >> /etc/sudoers.d/dderichswei
sed -i "s/^.*requiretty/#Defaults requiretty/" /etc/sudoers

/usr/bin/yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo
/usr/bin/yum install docker-ce -y
/usr/bin/systemctl enable docker

/usr/sbin/usermod  -a -G docker dderichswei
/usr/bin/yum -y install epel-release
/usr/bin/yum -y install python-pip
/usr/bin/pip install docker-py
%end
EOF

## create oneview_config.json

In [ ]:
cat << \EOF >> oneview_config.json
{
  "ip": "10.0.20.50",
  "credentials": {
    "userName": "user",
    "authLoginDomain": "local",
    "password": "password"
  },
  "image_streamer_ip": "10.0.20.54",
  "api_version": 1200
}


## if you don´t want to share your password, create a vault file

In [ ]:
cat << \EOF >> vault.yml
    ovpythonPW: !vault |
          $ANSIBLE_VAULT;1.1;AES256
          62623963656162326433363139363133653630393739356564343536323839326165396162383832
          3538333332396132383336623163396439303064313263330a326231646338316138393933313036
          31353965393039656661303661306561613633626539313030636538366532646233386333346130
          6239343739353364310a353636323539623863333632353063386538306565376234383736383532
          3062
    iloPW: !vault |
          $ANSIBLE_VAULT;1.1;AES256
          63386562623766626664663564643539663638393830373539326531383364313530623432623830
          3037613863633365613132356338333336323733376665640a373436636530303165643532393632
          63643864353733346538663038376630353465303930316434373762613261366561323930353066
          3434643164646439360a343363616335623062616331643136616439326134343334366138633838
          6330
    vcenterPW: !vault |
          $ANSIBLE_VAULT;1.1;AES256
          38353061353361326238643663306531383634333838303339386137663630653065656439643831
          3665393730363038313362633765396161373139396233660a663664323963306361383261323664
          36386461633935373266393434653434306162656633636264663964333966373863383232333666
          6337313530626238610a646562663463343136393363313866373163353538633165323161653231
          6639
    ansible_pass: !vault |
          $ANSIBLE_VAULT;1.1;AES256
          66343332313939356536613862616331343664633736393737646139383535626162333038373464
          3832336466353333643161323539646161396237633961360a643936333039386663306433343963
          63323966363732613131346564376530613165363730613939386562343038663837613564343334
          3639616336373237360a396133396131383439323438386530623430646665333934633164346161
          3734

EOF

In [ ]:
ansible-vault encrypt_string --vault-password-file .vault_password_file 'Password' --name 'ansible_pass'

In [ ]:
cat << \EOF > hosts
10.0.33.66 ansible_connection=ssh ansible_user=dderichswei ansible_password=Password

[all:vars]

EOF